In [38]:
import numpy as np
import matplotlib.pyplot as plt
import time

In [39]:
"""
Brute-force boolean matrix multiplication. Used as a benchmark for testing M4RM.

Parameters:
A, B: binary matricies 

Returns:
C: result of A * B 
"""
def boolean_matrix_multiplication(A, B):
    C = np.zeros((A.shape[0], B.shape[1]), dtype=int)
    for i in range(A.shape[0]):
        for j in range(B.shape[1]):
            for k in range(A.shape[1]):
                C[i, j] |= (A[i, k] & B[k, j])
    return C

In [40]:
"""
Multiply two binary matrices using the method of rour russians.

Parameters:
A, B: binary matricies 

Returns:
C: result of A * B 
"""
def four_russians(A, B):
    if A.shape[1] != B.shape[0]:
        raise ValueError(f"Matrix dimensions don't match for multiplication: A is {A.shape}, B is {B.shape}")
    
    if not np.all((A == 0) | (A == 1)) or not np.all((B == 0) | (B == 1)):
        raise ValueError("Input matrices must be binary")
    
    # multiplication is A (n x m) * B (m x p) = C (n x p)
    n = A.shape[0]
    m = A.shape[1]
    p = B.shape[1]
    
    k = max(1, int(np.log2(m)))
    
    C = np.zeros((n, p), dtype=int)
    
    for j in range(0, m, k):
        # Determine the actual block size, if k does not evenly divide m it may be smaller on the final iteration 
        block_size = min(k, m - j)
        
        num_combinations = 2**block_size
        row_sums = [np.zeros(p, dtype=int) for _ in range(num_combinations)]
        
        for r in range(1, num_combinations):
            for bit in range(block_size):
                if (r >> bit) & 1:
                    row_idx = j + bit
                    if row_idx < m: 
                        row_sums[r] = row_sums[r] | B[row_idx]

        
        # Extract the relevant block from A
        for i in range(n):
            
            a_block = A[i, j:j+block_size]
            
            # Convert the block to an index
            idx = 0
            for bit_idx, bit_val in enumerate(a_block):
                if bit_val:
                    idx |= (1 << bit_idx)
            
            # Update the result matrix with the precomputed block
            C[i] = C[i] | row_sums[idx]
    
    return C

In [41]:
'''
Helper function to compare the performance of the M4RM algorithm with the standard algorithm

Parameters: [max_size=100] largest size of the matrix to test
            [dn=10] increment size for the matrix size

Returns: None, prints graph of the performance comparison
'''
def test_four_russians(max_size=100, dn=10):
    time_four_russians = {}
    f4 = []
    t = []
    idx = []
    time_standard = {}
    n = 1
    while n < max_size:
        idx.append(n)
        A = np.random.randint(0, 2, (n, n))
        B = np.random.randint(0, 2, (n, n))

        start = time.time()
        result1 = four_russians(A, B)
        end = time.time()
        f4.append(end - start)
        time_four_russians[n] = end - start

        start = time.time()
        result2 = boolean_matrix_multiplication(A, B)
        end = time.time()
        t.append(end - start)
        time_standard[n] = end - start
        print(np.array_equal(result1, result2))
        n += dn

    plt.plot(idx, f4, label='M4RM')
    plt.plot(idx, t, label='Standard')

    plt.xlabel("Size of square matrix (n x n)")
    plt.ylabel("Time (seconds)")
    plt.title("Binary Matrix Multiplication using M4RM and the Standard Algorithm")
    plt.semilogy()
    plt.legend()
    plt.show()